In [3]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from yahoo_finance import Share
from urllib import urlopen

In [4]:
ticker = ['AAPL', 'MSFT', 'XOM']

url = 'http://www.reuters.com/finance/stocks/companyOfficers?symbol=' + ticker[0]
soup = bs(url)
soup.prettify()

u'<html>\n <body>\n  <p>\n   http://www.reuters.com/finance/stocks/companyOfficers?symbol=AAPL\n  </p>\n </body>\n</html>'

In [ ]:
bs.previousSibling()